In [5]:
# XML解析関数の修正
def parse_pubmed_data(xml_data):
    """PubMedから取得したXMLデータを解析"""
    root = ET.fromstring(xml_data)
    articles = []
    for article in root.findall(".//PubmedArticle"):
        pmid = article.find(".//PMID").text if article.find(".//PMID") is not None else "No PMID"
        title = article.find(".//ArticleTitle").text if article.find(".//ArticleTitle") is not None else "No Title"

        # First AuthorとLast Authorの取得
        first_author = "No First Author"
        first_affiliation = "No First Author Affiliation"
        last_author = "No Last Author"
        last_affiliation = "No Last Author Affiliation"

        authors = article.findall(".//Author")
        if authors:
            # First Author
            first = authors[0]
            first_fore_name = first.find("ForeName")
            first_last_name = first.find("LastName")
            if first_fore_name is not None and first_last_name is not None:
                first_author = f"{first_fore_name.text} {first_last_name.text}"
            first_affiliation_elem = first.find(".//AffiliationInfo/Affiliation")
            if first_affiliation_elem is not None:
                first_affiliation = first_affiliation_elem.text

            # Last Author
            last = authors[-1]
            last_fore_name = last.find("ForeName")
            last_last_name = last.find("LastName")
            if last_fore_name is not None and last_last_name is not None:
                last_author = f"{last_fore_name.text} {last_last_name.text}"
            last_affiliation_elem = last.find(".//AffiliationInfo/Affiliation")
            if last_affiliation_elem is not None:
                last_affiliation = last_affiliation_elem.text

        # その他の情報を取得
        author_list = ", ".join(
            f"{author.find('ForeName').text} {author.find('LastName').text}"
            for author in authors
            if author.find("ForeName") is not None and author.find("LastName") is not None
        ) if authors else "No Authors"
        journal = article.find(".//Title").text if article.find(".//Title") is not None else "No Journal"
        pub_date = article.find(".//PubDate/Year").text if article.find(".//PubDate/Year") is not None else "No Date"
        abstract_parts = [abstract.text for abstract in article.findall(".//AbstractText") if abstract.text]
        abstract = " ".join(abstract_parts) if abstract_parts else "No Abstract"
        link = f"https://pubmed.ncbi.nlm.nih.gov/{pmid}/"

        articles.append({
            "pmid": pmid,
            "title": title,
            "first_author": first_author,
            "first_affiliation": first_affiliation,
            "last_author": last_author,
            "last_affiliation": last_affiliation,
            "authors": author_list,
            "journal": journal,
            "pub_date": pub_date,
            "abstract": abstract,
            "link": link,
        })
    return articles

# メール本文のHTMLフォーマット修正版
def format_html_email(articles, keyword):
    """解析したデータをHTMLメール用にフォーマット"""
    content = f"<html><body><h2>Results for {keyword}</h2><hr>"
    for article in articles:
        content += f"""
        <p><strong>Title:</strong> {article['title']}</p>
        <p><strong>PMID:</strong> <a href="{article['link']}">{article['pmid']}</a></p>
        <p><strong>First Author:</strong> {article['first_author']} ({article['first_affiliation']})</p>
        <p><strong>Last Author:</strong> {article['last_author']} ({article['last_affiliation']})</p>
        <p><strong>Authors:</strong> {article['authors']}</p>
        <p><strong>Journal:</strong> {article['journal']} ({article['pub_date']})</p>
        <p><strong>Abstract:</strong> {article['abstract']}</p>
        <hr>
        """
    content += "</body></html>"
    return content

# メインループ（変更なし）
for keyword in keywords:
    ids = search_pubmed(keyword, start_date, end_date, api_key)
    if ids:
        xml_data = fetch_abstracts(ids, api_key)
        articles = parse_pubmed_data(xml_data)

        # Excelに保存
        save_to_excel(articles, sheet_name=keyword)

        # メール本文のフォーマット
        html_content = format_html_email(articles, keyword)

        # メール送信
        send_email(subject=f"PubMed Results for {keyword}", html_content=html_content)
        print(f"メール送信完了: {keyword}")
    else:
        print(f"No new articles for keyword: {keyword}")


メール送信完了: RNA splicing
メール送信完了: Glioma


In [1]:
!pip install openpyxl requests
